In [24]:
import gurobipy as gp
from gurobipy import GRB

[GRB](https://docs.gurobi.com/projects/optimizer/en/current/reference/python/grb.html) é uma classe que disponibiliza constantes para códigos numéricos para serem usadas na programação com a API do Gurobi.

## Problema de alocação de recursos - Resource Assignment Problem (RAP)

Considerando R um set de recursos: três possíveis funcionários

Considerando J um set de empregos, nos quais serão alocados os funcionários

In [25]:
R = ['Carlos', 'Joe', 'Monika']
J = ['Tester', 'JavaDeveloper', 'Architect']

A habilidade de cada funcionário em cada área é:

In [26]:
combinations, scores = gp.multidict({
    ('Carlos', 'Tester'): 53,
    ('Carlos', 'JavaDeveloper'): 27,
    ('Carlos', 'Architect'): 13,
    ('Joe', 'Tester'): 80,
    ('Joe', 'JavaDeveloper'): 47,
    ('Joe', 'Architect'): 67,
    ('Monika', 'Tester'): 53,
    ('Monika', 'JavaDeveloper'): 73,
    ('Monika', 'Architect'): 47
})

In [27]:
combinations

<gurobi.tuplelist (9 tuples, 2 values each):
 ( Carlos , Tester        )
 ( Carlos , JavaDeveloper )
 ( Carlos , Architect     )
 ( Joe    , Tester        )
 ( Joe    , JavaDeveloper )
 ( Joe    , Architect     )
 ( Monika , Tester        )
 ( Monika , JavaDeveloper )
 ( Monika , Architect     )
>

In [28]:
scores

{('Carlos', 'Tester'): 53,
 ('Carlos', 'JavaDeveloper'): 27,
 ('Carlos', 'Architect'): 13,
 ('Joe', 'Tester'): 80,
 ('Joe', 'JavaDeveloper'): 47,
 ('Joe', 'Architect'): 67,
 ('Monika', 'Tester'): 53,
 ('Monika', 'JavaDeveloper'): 73,
 ('Monika', 'Architect'): 47}

Necessário variável de decisão. Nesse caso, é binária (x1,1 = 0 Carlos não foi escolhido para a posição de Tester, e vice-versa)


In [29]:
m = gp.Model('RAP')
x = m.addVars(combinations, name="assign") # variáveis de decisão

As restrições, considerando que deve-se ter uma pessoa por cargo, e que uma pessoa não pode ser alocada em mais de um cargo:

In [30]:
jobs = m.addConstrs((x.sum('*',j) == 1 for j in J), name='job')
resources = m.addConstrs((x.sum(r,'*') <= 1 for r in R), name='resource')

A função objetivo:

In [31]:
m.setObjective(x.prod(scores), GRB.MAXIMIZE)

In [32]:
m.write('RAP.lp')
m.optimize()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.6.0 24G90)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6 rows, 9 columns and 18 nonzeros
Model fingerprint: 0xb343b6eb
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 6 rows, 9 columns, 18 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.6000000e+32   1.800000e+31   4.600000e+02      0s
       5    1.9300000e+02   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.930000000e+02


getVars pra recuperar as variáveis do modelo:

In [34]:
for v in m.getVars():
    if v.x > 1e-6:
        print(v.varName, v.x)

# Display optimal total matching score
print('Total matching score: ', m.objVal)

assign[Carlos,Tester] 1.0
assign[Joe,Architect] 1.0
assign[Monika,JavaDeveloper] 1.0
Total matching score:  193.0


# RAP com constraint de custo

Considerando Carlos 1k, Joe 2k e Monika 3k, com budget de 5k

In [36]:
R = ['Carlos', 'Joe', 'Monika']
J = ['Tester', 'JavaDeveloper', 'Architect']

combinations, scores, costs = gp.multidict({
    ('Carlos', 'Tester'): [53, 1],
    ('Carlos', 'JavaDeveloper'): [27, 1],
    ('Carlos', 'Architect'): [13,1],
    ('Joe', 'Tester'): [80, 2],
    ('Joe', 'JavaDeveloper'): [47, 2],
    ('Joe', 'Architect'): [67, 2],
    ('Monika', 'Tester'): [53, 3] ,
    ('Monika', 'JavaDeveloper'): [73, 3],
    ('Monika', 'Architect'): [47, 3]
})

budget = 5

m = gp.Model('RAP2')

Variável gap que indica se alguma posição não foi cumprida

In [37]:
x = m.addVars(combinations, vtype=GRB.BINARY, name="assign")

g = m.addVars(J, name="gap")

jobs = m.addConstrs((x.sum('*',j) + g[j]  == 1 for j in J), name='job')
resources = m.addConstrs((x.sum(r,'*') <= 1 for r in R), name='resource')
budget = m.addConstr((x.prod(costs) <= budget), name='budget')

Penalizando não preencher uma posição

In [38]:
M = 101
m.setObjective(x.prod(scores) - M*g.sum(), GRB.MAXIMIZE)
m.optimize()
total_matching_score = 0
for r, j in combinations:
    if x[r, j].x > 1e-6:
        print(x[r, j].varName, x[r, j].x) 
        total_matching_score += scores[r, j]*x[r, j].x

print('Total matching score: ', total_matching_score)  

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.6.0 24G90)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 12 columns and 30 nonzeros
Model fingerprint: 0xa1231a12
Variable types: 3 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Presolve time: 0.00s
Presolved: 7 rows, 12 columns, 30 nonzeros
Variable types: 0 continuous, 12 integer (12 binary)
Found heuristic solution: objective 52.0000000

Root relaxation: objective 1.350000e+02, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  135.00000    0    2   52.00000  135.00000   160%     -    0s
     0     0     cutoff    0        52.000

In [39]:
m.dispose()
gp.disposeDefaultEnv()

Freeing default Gurobi environment


... do [exemplo oficial](https://colab.research.google.com/github/Gurobi/modeling-examples/blob/master/milp_tutorial/introduction_to_modeling.ipynb#scrollTo=piCWvWr7Psz6)